<a href="https://colab.research.google.com/github/Aasthajoshay/ScientificPaperSummarization-QnA/blob/main/ScientificPaperSummarization_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: install pypdf2, numpy,fitz,faiss

!pip install pypdf2 numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.2 MB/s eta 0:00:00


In [4]:

!pip install pymupdf

# Now, try importing fitz again
import fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.4 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata
from openai import OpenAI  # Groq uses OpenAI-compatible API
from typing import List, Dict, Any
import re
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Re-importing PDFCorpusManager from the previous cell to ensure it's available
# when ScientificQASystem is defined in this new combined cell.
# Alternatively, you could combine all classes into a single cell.
from pathlib import Path
import json
from datetime import datetime
import faiss


class PDFCorpusManager:
    def __init__(self, corpus_directory="./pdf_corpus", embeddings_path="./embeddings"):
        self.corpus_directory = Path(corpus_directory)
        self.embeddings_path = Path(embeddings_path)
        self.corpus_directory.mkdir(exist_ok=True)
        self.embeddings_path.mkdir(exist_ok=True)

        # Initialize embedding model
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Metadata storage
        self.metadata = {}
        self.chunks = []
        self.index = None

    def upload_pdf(self, pdf_file_path: str, metadata: Dict = None):
        """Upload and process a single PDF file"""
        pdf_path = Path(pdf_file_path)

        if not pdf_path.exists():
            potential_path = Path("/content") / pdf_path.name
            if potential_path.exists():
                 pdf_path = potential_path
                 print(f"File not found at {pdf_file_path}, found at {potential_path}")
            else:
                 raise FileNotFoundError(f"PDF file not found at {pdf_file_path} or {potential_path}")


        # Extract text from PDF
        text_content = self._extract_text_from_pdf(pdf_path)

        # Generate unique document ID
        doc_id = f"doc_{len(self.metadata)}_{pdf_path.stem}"

        # Store metadata
        self.metadata[doc_id] = {
            "filename": pdf_path.name,
            "upload_date": datetime.now().isoformat(),
            "file_path": str(pdf_path),
            "text_length": len(text_content),
            "metadata": metadata or {}
        }

        # Chunk the document
        doc_chunks = self._chunk_document(text_content, doc_id)
        self.chunks.extend(doc_chunks)

        print(f"✅ Uploaded and processed: {pdf_path.name}")
        print(f"   - Generated {len(doc_chunks)} chunks")
        print(f"   - Total corpus size: {len(self.chunks)} chunks")

        return doc_id

    def _extract_text_from_pdf(self, pdf_path: Path) -> str:
        """Extracts text from a PDF file using PyMuPDF (fitz)."""
        text = ""
        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    text += page.get_text()
            print(f"Successfully extracted text from {pdf_path.name}")
        except Exception as e:
            print(f"Error extracting text from {pdf_path.name}: {e}")
            text = ""
        return text

    def _chunk_document(self, text: str, doc_id: str, chunk_size: int = 512, overlap: int = 64) -> List[Dict]:
        """Split document into chunks with metadata"""
        words = text.split()
        chunks = []

        for i in range(0, len(words), chunk_size - overlap):
            chunk_text = " ".join(words[i:i + chunk_size])

            chunk_data = {
                "text": chunk_text,
                "doc_id": doc_id,
                "chunk_index": len(chunks),
                "start_word": i,
                "end_word": min(i + chunk_size, len(words))
            }
            chunks.append(chunk_data)

        return chunks

    def build_corpus_index(self):
        """Build FAISS index for the entire corpus"""
        if not self.chunks:
            raise ValueError("No documents in corpus. Upload PDFs first.")

        print("🔄 Building corpus embeddings...")

        chunk_texts = [chunk["text"] for chunk in self.chunks]

        embeddings = self.embedding_model.encode(chunk_texts, show_progress_bar=True)

        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings.astype('float32'))

        print(f"✅ Built corpus index with {len(chunk_texts)} chunks")

        self._save_corpus()

    def _save_corpus(self):
        """Save corpus index and metadata to disk"""
        faiss.write_index(self.index, str(self.embeddings_path / "corpus.index"))

        corpus_data = {
            "metadata": self.metadata,
            "chunks": self.chunks,
            "embedding_model": "all-MiniLM-L6-v2",
            "created_date": datetime.now().isoformat()
        }

        with open(self.embeddings_path / "corpus_data.json", "w") as f:
            json.dump(corpus_data, f, indent=2)

        print(f"💾 Saved corpus to {self.embeddings_path}")

    def load_corpus(self):
        """Load existing corpus from disk"""
        index_path = self.embeddings_path / "corpus.index"
        data_path = self.embeddings_path / "corpus_data.json"

        if not (index_path.exists() and data_path.exists()):
            print("⚠️  No existing corpus found. Upload PDFs and build index first.")
            return False

        self.index = faiss.read_index(str(index_path))

        with open(data_path, "r") as f:
            corpus_data = json.load(f)

        self.metadata = corpus_data["metadata"]
        self.chunks = corpus_data["chunks"]

        print(f"✅ Loaded corpus with {len(self.chunks)} chunks from {len(self.metadata)} documents")
        return True

    def search_corpus(self, query: str, k: int = 5) -> List[Dict]:
        """Search corpus for relevant chunks"""
        if self.index is None:
            raise ValueError("Corpus index not built. Call build_corpus_index() first.")

        query_embedding = self.embedding_model.encode([query])

        distances, indices = self.index.search(query_embedding.astype('float32'), k)

        results = []
        for i, idx in enumerate(indices[0]):
            chunk = self.chunks[idx].copy()
            chunk["similarity_score"] = float(distances[0][i])
            chunk["document_info"] = self.metadata[chunk["doc_id"]]
            results.append(chunk)

        return results

    def get_corpus_stats(self):
        """Get statistics about the corpus"""
        return {
            "total_documents": len(self.metadata),
            "total_chunks": len(self.chunks),
            "documents": [
                {
                    "doc_id": doc_id,
                    "filename": info["filename"],
                    "upload_date": info["upload_date"],
                    "text_length": info["text_length"]
                }
                for doc_id, info in self.metadata.items()
            ]
        }


class ScientificQASystem:
    def __init__(self, corpus_manager: PDFCorpusManager):
        self.corpus_manager = corpus_manager

        # Get Groq API key from Colab secrets
        try:
            groq_api_key = userdata.get('GROQ_API_KEY')
        except Exception as e:
            raise ValueError("GROQ_API_KEY not found in Colab secrets. Please add it in the secrets panel.")

        # Initialize Groq client (uses OpenAI-compatible interface)
        self.client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=groq_api_key
        )

        print("✅ Groq API client initialized successfully")

    def build_few_shot_prompt(self, user_question: str, retrieved_chunks: List[Dict]) -> str:
        """Build few-shot prompt with retrieved context from corpus"""

        context_parts = []
        for chunk in retrieved_chunks:
            doc_info = chunk["document_info"]
            context_parts.append(f"[From {doc_info['filename']}]: {chunk['text']}")

        combined_context = "\n\n".join(context_parts)

        prompt = f"""You are a research assistant analyzing a corpus of scientific journal articles. Given a question and relevant context from multiple papers, provide a concise, accurate answer based only on the provided context.

Example 1:
Question: What methodology was used to collect data about AI tool usage in the study?
Context: [From paper1.pdf]: Data for the study were collected using a structured questionnaire distributed during the final team presentation of the programming project. The questionnaire was structured to comprehensively evaluate the adoption and application of AI tools during the programming project, following the Technology Acceptance Model.
Answer: The researchers used a structured questionnaire distributed during final team presentations, following the Technology Acceptance Model framework to evaluate AI tool adoption and application.

Example 2:
Question: What was the adoption rate of AI tools among students in the programming project?
Context: [From paper1.pdf]: Of the 38 participants in this study, 34 used AI tools during the programming project, resulting in an adoption rate of 89.4%. The remaining four participants all reported that they did not use AI tools, citing either a lack of need or personal reservations.
Answer: The AI tool adoption rate was 89.4%, with 34 out of 38 participants using AI tools during the programming project.
Example 3:
Question: Which development phase had the highest AI tool usage?",
Context: [From paper1.pdf]: The Coding & Testing phase had the highest adoption rate, with 33 participants using AI tools and only 5 opting out. CHATGPT was used by all students who used AI tools, regardless of the phase.",
Answer: The Coding & Testing phase had the highest AI tool usage with 33 out of 38 participants using AI tools.

Now answer the following:

Question: {user_question}
Context: {combined_context}

Answer:"""

        return prompt

    def answer_question(self, question: str, num_chunks: int = 5) -> Dict[str, Any]:
        """Answer a question using the corpus with Groq API"""

        # Step 1: Retrieve relevant chunks from corpus
        print(f"🔍 Searching corpus for: {question}")
        retrieved_chunks = self.corpus_manager.search_corpus(question, k=num_chunks)

        # Step 2: Build few-shot prompt
        prompt = self.build_few_shot_prompt(question, retrieved_chunks)

        # Step 3: Generate answer using Groq API
        print("🤖 Generating answer with Groq...")
        try:
            response = self.client.chat.completions.create(
                model="llama3-8b-8192",  # Fast Groq model
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0.1,
                top_p=1,
                stream=False
            )

            answer = response.choices[0].message.content.strip()

        except Exception as e:
            print(f"❌ Error with Groq API: {e}")
            answer = "Sorry, I encountered an error while generating the answer."

        # Return comprehensive result
        return {
            "question": question,
            "answer": answer,
            "sources": [
                {
                    "document": chunk["document_info"]["filename"],
                    "similarity_score": chunk["similarity_score"],
                    "text_snippet": chunk["text"][:200] + "..."
                }
                for chunk in retrieved_chunks
            ],
            "num_sources": len(retrieved_chunks),
            "model_used": "llama3-8b-8192 (Groq)"
        }

    def batch_answer_questions(self, questions: List[str]) -> List[Dict[str, Any]]:
        """Answer multiple questions in batch"""
        results = []

        for i, question in enumerate(questions, 1):
            print(f"\n📝 Processing question {i}/{len(questions)}")
            result = self.answer_question(question)
            results.append(result)

        return results


class QAMetrics:
    def __init__(self):
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

    def exact_match(self, predicted: str, ground_truth: str) -> float:
        """Exact Match Score - strict character-level matching"""
        predicted = predicted.strip().lower()
        ground_truth = ground_truth.strip().lower()
        return 1.0 if predicted == ground_truth else 0.0

    def f1_score_qa(self, predicted: str, ground_truth: str) -> float:
        """Token-level F1 Score for QA"""
        def normalize_text(text):
            text = re.sub(r'[^\w\s]', '', text.lower())
            return set(text.split())

        pred_tokens = normalize_text(predicted)
        truth_tokens = normalize_text(ground_truth)

        if not truth_tokens:
            return 1.0 if not pred_tokens else 0.0

        common_tokens = pred_tokens.intersection(truth_tokens)

        if not common_tokens:
            return 0.0

        precision = len(common_tokens) / len(pred_tokens) if pred_tokens else 0.0
        recall = len(common_tokens) / len(truth_tokens)

        if precision + recall == 0:
            return 0.0

        f1 = 2 * (precision * recall) / (precision + recall)
        return f1

    def semantic_similarity(self, predicted: str, ground_truth: str) -> float:
        """Semantic similarity using sentence embeddings"""
        pred_embedding = self.sentence_model.encode([predicted])
        truth_embedding = self.sentence_model.encode([ground_truth])

        similarity = cosine_similarity(pred_embedding, truth_embedding)[0][0]
        return max(0.0, similarity)

    def answer_relevancy(self, question: str, answer: str) -> float:
        """Measure how relevant the answer is to the question"""
        question_embedding = self.sentence_model.encode([question])
        answer_embedding = self.sentence_model.encode([answer])

        relevancy = cosine_similarity(question_embedding, answer_embedding)[0][0]
        return max(0.0, relevancy)


class QAEvaluator:
    def __init__(self, qa_system: ScientificQASystem):
        self.qa_system = qa_system
        self.evaluation_data = []

    def create_evaluation_dataset(self):
        """Create ground truth Q&A pairs for evaluation"""
        evaluation_questions = [
            {
                "question": "What methodology was used to collect data in the AI tools study?",
                "ground_truth": "A structured questionnaire was distributed during final team presentations, following the Technology Acceptance Model framework.",
                "category": "methodology"
            },
            {
                "question": "What was the adoption rate of AI tools among students?",
                "ground_truth": "89.4% adoption rate, with 34 out of 38 participants using AI tools.",
                "category": "statistics"
            },
            {
                "question": "Which development phase had the highest AI tool usage?",
                "ground_truth": "The Coding & Testing phase had the highest usage with 33 out of 38 participants.",
                "category": "findings"
            }
        ]

        self.evaluation_data = evaluation_questions
        return evaluation_questions

    def evaluate_retrieval_quality(self, question: str, ground_truth_docs: List[str], k: int = 5) -> Dict:
        """Evaluate retrieval component performance"""
        retrieved_chunks = self.qa_system.corpus_manager.search_corpus(question, k=k)
        retrieved_doc_names = [chunk["document_info"]["filename"] for chunk in retrieved_chunks]

        # Calculate Recall@K
        relevant_retrieved = sum(1 for doc in ground_truth_docs if doc in retrieved_doc_names)
        recall_at_k = relevant_retrieved / len(ground_truth_docs) if ground_truth_docs else 0.0

        # Calculate Mean Reciprocal Rank (MRR)
        mrr = 0.0
        for i, doc_name in enumerate(retrieved_doc_names):
            if doc_name in ground_truth_docs:
                mrr = 1.0 / (i + 1)
                break

        return {
            "recall_at_k": recall_at_k,
            "mrr": mrr,
            "retrieved_docs": retrieved_doc_names,
            "num_relevant_retrieved": relevant_retrieved
        }

    def run_comprehensive_evaluation(self):
        """Run complete evaluation of the QA system"""

        print("🔍 Starting Comprehensive Evaluation...")

        metrics = QAMetrics()
        results = {
            "exact_match_scores": [],
            "f1_scores": [],
            "semantic_similarity_scores": [],
            "answer_relevancy_scores": [],
            "retrieval_metrics": [],
            "detailed_results": []
        }

        evaluation_data = self.create_evaluation_dataset()

        for i, eval_item in enumerate(evaluation_data):
            print(f"\n📝 Evaluating question {i+1}/{len(evaluation_data)}")

            question = eval_item["question"]
            ground_truth = eval_item["ground_truth"]

            # Generate answer
            qa_result = self.qa_system.answer_question(question)
            predicted_answer = qa_result["answer"]

            # Calculate metrics
            em_score = metrics.exact_match(predicted_answer, ground_truth)
            f1_score = metrics.f1_score_qa(predicted_answer, ground_truth)
            semantic_score = metrics.semantic_similarity(predicted_answer, ground_truth)
            relevancy_score = metrics.answer_relevancy(question, predicted_answer)

            # Store results
            results["exact_match_scores"].append(em_score)
            results["f1_scores"].append(f1_score)
            results["semantic_similarity_scores"].append(semantic_score)
            results["answer_relevancy_scores"].append(relevancy_score)

            detailed_result = {
                "question": question,
                "predicted_answer": predicted_answer,
                "ground_truth": ground_truth,
                "exact_match": em_score,
                "f1_score": f1_score,
                "semantic_similarity": semantic_score,
                "answer_relevancy": relevancy_score,
                "category": eval_item["category"]
            }
            results["detailed_results"].append(detailed_result)

            print(f"   EM: {em_score:.3f} | F1: {f1_score:.3f} | Semantic: {semantic_score:.3f}")

        # Calculate averages
        avg_results = {
            "avg_exact_match": np.mean(results["exact_match_scores"]),
            "avg_f1_score": np.mean(results["f1_scores"]),
            "avg_semantic_similarity": np.mean(results["semantic_similarity_scores"]),
            "avg_answer_relevancy": np.mean(results["answer_relevancy_scores"])
        }

        return results, avg_results

    def generate_evaluation_report(self, results: Dict, avg_results: Dict):
        """Generate comprehensive evaluation report"""

        print("\n" + "="*60)
        print("📊 SCIENTIFIC QA SYSTEM EVALUATION REPORT")
        print("="*60)

        print(f"\n🎯 Overall Performance Metrics:")
        print(f"   • Average Exact Match:      {avg_results['avg_exact_match']:.3f}")
        print(f"   • Average F1 Score:         {avg_results['avg_f1_score']:.3f}")
        print(f"   • Average Semantic Sim:     {avg_results['avg_semantic_similarity']:.3f}")
        print(f"   • Average Answer Relevancy: {avg_results['avg_answer_relevancy']:.3f}")

        # Performance by category
        categories = {}
        for result in results["detailed_results"]:
            cat = result["category"]
            if cat not in categories:
                categories[cat] = {"f1_scores": [], "semantic_scores": []}
            categories[cat]["f1_scores"].append(result["f1_score"])
            categories[cat]["semantic_scores"].append(result["semantic_similarity"])

        print(f"\n📈 Performance by Question Category:")
        for category, scores in categories.items():
            avg_f1 = np.mean(scores["f1_scores"])
            avg_semantic = np.mean(scores["semantic_scores"])
            print(f"   • {category.title()}: F1={avg_f1:.3f}, Semantic={avg_semantic:.3f}")

        # Best and worst performing questions
        sorted_results = sorted(results["detailed_results"],
                              key=lambda x: x["f1_score"], reverse=True)

        print(f"\n✅ Best Performing Question:")
        best = sorted_results[0]
        print(f"   Q: {best['question'][:80]}...")
        print(f"   F1: {best['f1_score']:.3f}")

        print(f"\n❌ Worst Performing Question:")
        worst = sorted_results[-1]
        print(f"   Q: {worst['question'][:80]}...")
        print(f"   F1: {worst['f1_score']:.3f}")

        return {
            "overall_metrics": avg_results,
            "category_performance": categories,
            "best_question": best,
            "worst_question": worst
        }

In [6]:
def answer_question(self, question: str, num_chunks: int = 5) -> Dict[str, Any]:
        """Answer a question using the corpus with Groq API"""

        # Step 1: Retrieve relevant chunks from corpus
        print(f"🔍 Searching corpus for: {question}")
        retrieved_chunks = self.corpus_manager.search_corpus(question, k=num_chunks)

        # Step 2: Build few-shot prompt
        prompt = self.build_few_shot_prompt(question, retrieved_chunks)

        # Step 3: Generate answer using Groq API
        print("🤖 Generating answer with Groq...")
        try:
            response = self.client.chat.completions.create(
                model="llama3-8b-8192",  # Fast Groq model
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0.1,
                top_p=1,
                stream=False
            )

            answer = response.choices[0].message.content.strip()

        except Exception as e:
            print(f"❌ Error with Groq API: {e}")
            answer = "Sorry, I encountered an error while generating the answer."

        # Return comprehensive result
        return {
            "question": question,
            "answer": answer,
            "sources": [
                {
                    "document": chunk["document_info"]["filename"],
                    "similarity_score": chunk["similarity_score"],
                    "text_snippet": chunk["text"][:200] + "..."
                }
                for chunk in retrieved_chunks
            ],
            "num_sources": len(retrieved_chunks),
            "model_used": "llama3-8b-8192 (Groq)"
        }


In [8]:
import os
from google.colab import userdata

# Retrieve the Groq API Key securely from Colab's Userdata secrets
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')

    # Set the environment variable for the Groq client to use
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY

except SecretNotFoundError:
    print("Error: Secret 'GROQ_API_KEY' not found in Colab secrets.")
    print("Please add your Groq API Key to the Colab secrets manager")
    print("Click the key icon in the left sidebar.")
    GROQ_API_KEY = None # Ensure GROQ_API_KEY is None if not found

In [9]:
# Check if the API key is loaded
if not GROQ_API_KEY:
    print("Error: GROQ_API_KEY not found. Please set it in your environment or .env file.")
else:
    print("GROQ_API_KEY loaded successfully.")


GROQ_API_KEY loaded successfully.


In [10]:
# Initialize the system
corpus_manager = PDFCorpusManager(corpus_directory="./scientific_papers")
qa_system = ScientificQASystem(corpus_manager)

# Upload multiple PDFs to build corpus
pdf_files = [
    "2025ECSEEGenerativeAIinStudentSoftwareDevelopmentProjects.pdf",
    "2406.00560v1.pdf",
    "Elsevier_Article__elsarticle__Template1.pdf",
    "GenAI___Productivity.pdf", # Added missing comma here
    "applsci-15-05785.pdf"
]

print("📚 Building Scientific Paper Corpus...")
for pdf_file in pdf_files:
    pdf_path = Path(pdf_file) # Define pdf_path here

    if not pdf_path.exists():
        # Adjusted the default path...
        potential_path = Path("/content") / pdf_path.name
        if potential_path.exists():
             pdf_path = potential_path
             print(f"File not found at {pdf_file}, found at {potential_path}") # Changed f-string to use pdf_file
        else:
             raise FileNotFoundError(f"PDF file not found at {pdf_file} or {potential_path}") # Changed f-string to use pdf_file

    # Upload the PDF file
    corpus_manager.upload_pdf(str(pdf_path)) # Call upload_pdf with the path

# Build the corpus index
corpus_manager.build_corpus_index()

# Display corpus statistics
stats = corpus_manager.get_corpus_stats()
print(f"\n📊 Corpus Statistics:")
print(f"   - Total Documents: {stats['total_documents']}")
print(f"   - Total Chunks: {stats['total_chunks']}")

# Example scientist queries
questions = [
    "What challenges did students face when using AI tools in software development?",
    "How effective are AI tools in improving coding productivity?",
    "What methodologies were used to evaluate student performance?",
    "What are the main factors influencing AI tool adoption in education?"
]

print("\n🔬 Answering Scientific Questions...")
for question in questions:
    result = qa_system.answer_question(question)

    print(f"\n❓ Question: {result['question']}")
    print(f"✅ Answer: {result['answer']}")
    print(f"📄 Sources: {result['num_sources']} documents")
    for source in result['sources'][:2]:  # Show top 2 sources
        print(f"   - {source['document']} (similarity: {source['similarity_score']:.3f})")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Groq API client initialized successfully
📚 Building Scientific Paper Corpus...
Successfully extracted text from 2025ECSEEGenerativeAIinStudentSoftwareDevelopmentProjects.pdf
✅ Uploaded and processed: 2025ECSEEGenerativeAIinStudentSoftwareDevelopmentProjects.pdf
   - Generated 21 chunks
   - Total corpus size: 21 chunks
Successfully extracted text from 2406.00560v1.pdf
✅ Uploaded and processed: 2406.00560v1.pdf
   - Generated 16 chunks
   - Total corpus size: 37 chunks
Successfully extracted text from Elsevier_Article__elsarticle__Template1.pdf
✅ Uploaded and processed: Elsevier_Article__elsarticle__Template1.pdf
   - Generated 0 chunks
   - Total corpus size: 37 chunks
Successfully extracted text from GenAI___Productivity.pdf
✅ Uploaded and processed: GenAI___Productivity.pdf
   - Generated 16 chunks
   - Total corpus size: 53 chunks
Successfully extracted text from applsci-15-05785.pdf
✅ Uploaded and processed: applsci-15-05785.pdf
   - Generated 0 chunks
   - Total corpus size: 53 

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Built corpus index with 53 chunks
💾 Saved corpus to embeddings

📊 Corpus Statistics:
   - Total Documents: 5
   - Total Chunks: 53

🔬 Answering Scientific Questions...
🔍 Searching corpus for: What challenges did students face when using AI tools in software development?
🤖 Generating answer with Groq...

❓ Question: What challenges did students face when using AI tools in software development?
✅ Answer: What challenges did students face when using AI tools in software development?

According to the context, students faced several challenges when using AI tools in software development, including:

* Inaccurate results: Students reported encountering inaccurate results when using AI tools for tasks such as code generation, debugging, and comprehension.
* Missing context: Students mentioned that AI tools often lacked context or failed to understand domain-specific frameworks, leading to incorrect outputs.
* Limitations in handling complex code bases: Students noted that AI tools struggle

In [11]:
# prompt: tell the relevance accuracy of this model

evaluator = QAEvaluator(qa_system)

# Run the evaluation
results, avg_results = evaluator.run_comprehensive_evaluation()

# Generate and print the report
evaluation_report = evaluator.generate_evaluation_report(results, avg_results)

# The 'Answer Relevancy' metric from the report is a measure of how relevant the generated answer is to the question,
# based on semantic similarity. While not a direct "accuracy" measure against a ground truth answer's content,
# it indicates the model's ability to produce an answer that stays on topic.
print(f"\n✨ Relevance Accuracy (Average Answer Relevancy Score): {avg_results['avg_answer_relevancy']:.3f}")

# You could also consider semantic similarity to ground truth as a form of accuracy in terms of meaning
print(f"✨ Semantic Accuracy (Average Semantic Similarity Score): {avg_results['avg_semantic_similarity']:.3f}")


🔍 Starting Comprehensive Evaluation...

📝 Evaluating question 1/3
🔍 Searching corpus for: What methodology was used to collect data in the AI tools study?
🤖 Generating answer with Groq...
   EM: 0.000 | F1: 0.103 | Semantic: 0.390

📝 Evaluating question 2/3
🔍 Searching corpus for: What was the adoption rate of AI tools among students?
🤖 Generating answer with Groq...
   EM: 0.000 | F1: 0.606 | Semantic: 0.806

📝 Evaluating question 3/3
🔍 Searching corpus for: Which development phase had the highest AI tool usage?
🤖 Generating answer with Groq...
   EM: 0.000 | F1: 0.556 | Semantic: 0.629

📊 SCIENTIFIC QA SYSTEM EVALUATION REPORT

🎯 Overall Performance Metrics:
   • Average Exact Match:      0.000
   • Average F1 Score:         0.422
   • Average Semantic Sim:     0.608
   • Average Answer Relevancy: 0.795

📈 Performance by Question Category:
   • Methodology: F1=0.103, Semantic=0.390
   • Statistics: F1=0.606, Semantic=0.806
   • Findings: F1=0.556, Semantic=0.629

✅ Best Performing Qu